In [43]:
import json
import datasets
import pandas as pd
import numpy as np
from pprint import pprint
from tqdm.notebook import tqdm
from collections import Counter
from datasets import load_dataset
from time import strftime, localtime

pd.set_option('display.max_colwidth', 200)

In [2]:
# ! pip install fsspec=2023.4.0

In [3]:
# dataset = load_dataset('alsubari/Israel-palestine-war')

In [4]:
# print(dataset)

# Exploratory Data Analysis

### Stats about videos

In [5]:
df_metadata = pd.read_csv('data/alsubari_videos_metadata.csv')

In [6]:
df_metadata.head()

,Unnamed: 0,id,publish_channel,title,start_duration,end_duration,lengthSeconds,total_comments,total_unique_users
0,0,FUzLx1l37RE,NBCNews,Blinken reiterates Israel support but urges re...,2023-11-03 19:38:34,2023-11-04 17:38:34,241,25,21
1,1,PcQaG4sC9BM,NBCNews,Full Special Report: Israel declares war after...,2023-10-07 19:42:03,2023-11-04 12:38:36,1484,21905,14860
2,2,X2t6NyGP1-w,NBCNews,Hamas official claims it's their 'legal right'...,2023-11-03 19:49:36,2023-11-04 19:46:34,335,1853,1170
3,3,cd24N0tF-jY,NBCNews,FBI director warns of growing domestic threat ...,2023-11-01 19:50:24,2023-11-04 18:50:41,143,987,670
4,4,ICR5ml2YPkI,NBCNews,Israel’s president: 23-year-old Israeli hostag...,2023-10-31 19:51:23,2023-11-04 19:44:35,162,5275,3475


In [7]:
df_metadata['start_duration'] = pd.to_datetime(df_metadata['start_duration'])
df_metadata['end_duration'] = pd.to_datetime(df_metadata['end_duration'])

In [8]:
print("Number of videos:", len(df_metadata))
print("Number of channels:", len(df_metadata.publish_channel.unique()))
print("Total duration:", sum(df_metadata.lengthSeconds), "seconds")
print("Total number of comments:", sum(df_metadata.total_comments))
print("Total number of unique users:", sum(df_metadata.total_unique_users))

first_post = min(df_metadata.start_duration)
last_post = max(df_metadata.start_duration)
print('\n', "Time interval:", first_post, 'to', last_post, '(totaling', (last_post - first_post).days, 'days)')

Number of videos: 310
Number of channels: 6
Total duration: 134468 seconds
Total number of comments: 1028346
Total number of unique users: 689753

 Time interval: 2023-10-04 22:01:46 to 2023-11-06 19:42:46 (totaling 32 days)


### Stats about comments

In [9]:
df_comments = pd.read_csv('data/alsubari_videos_comments.csv')

In [10]:
df_comments.head()

,Unnamed: 0,author,channel,cid,heart,reply,text,time,time_parsed,votes,video_id
0,0,@LMLewis,UCNUa3f8nwYg-Kn1v4u5wrtQ,UgxbXce9xz7a27nVOyl4AaABAg,False,False,Blinken deserves to be known as the Butcher of...,2 hours ago,1.699120e+09,0,FUzLx1l37RE
1,1,@lovellewilliams5531,UCm9ztdJUAMpUNipdNxAHN_g,Ugx7aiPfji7zYkmV1k54AaABAg,False,False,The United States are supporting the killing o...,4 hours ago,1.699112e+09,0,FUzLx1l37RE
2,2,@noryaminmatariffin3029,UCCyvX_gd5zANsFYXnDBpuXw,UgywKu6FT8TWbssFsnJ4AaABAg,False,False,"So why hospitals, ambulance, civiliance, refug...",16 hours ago,1.699069e+09,0,FUzLx1l37RE
3,3,@Not_Today___,UCmicKai8OhhPUpOfG3VTFvA,UgwXQG7Qt7HVKsSTC7F4AaABAg,False,False,I'm an American. I think this war started on O...,1 day ago,1.699040e+09,0,FUzLx1l37RE
4,4,@benzo2632,UCA0boZvrgR4mbnQ2Bi65QKQ,UgxnJo7jgnD3Qcg6aAB4AaABAg,False,False,"WAR CRIMES SUPORTED BY USA ,,,,",1 day ago,1.699040e+09,0,FUzLx1l37RE


In [11]:
epoch_to_date = lambda d: strftime('%Y-%m-%d %H:%M:%S', localtime(d))
df_comments['date'] = pd.to_datetime(df_comments['time_parsed'].apply(epoch_to_date))

In [12]:
print("Number of comments:", len(df_comments))
print("Total number of unique posters:", len(df_comments.author.unique()))
print("Average number of comments per video:", int(df_comments['video_id'].value_counts().mean()))

first_post = min(df_comments.date)
last_post = max(df_comments.date)
print('\n', "Time interval:", first_post, 'to', last_post, '(totaling', (last_post - first_post).days, 'days)')

Number of comments: 1028346
Total number of unique posters: 457373
Average number of comments per video: 3317

 Time interval: 2023-10-04 23:01:46 to 2023-11-07 01:41:49 (totaling 33 days)


# Sentiment Analysis

In [13]:
from transformers import pipeline

In [14]:
# No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english 
sentiment_pipeline = pipeline("sentiment-analysis", device="cuda")
data = ["I love you", "I hate you"]
sentiment_pipeline(data)

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


[{'label': 'POSITIVE', 'score': 0.9998656511306763},
 {'label': 'NEGATIVE', 'score': 0.9991129040718079}]

In [15]:
df_comments['video_id'].value_counts()

video_id
4idQbwsvtUo    128093
6FVUxvp6Ah0     26252
svOc4Ki68_U     24391
FXrVtabZggI     23255
PcQaG4sC9BM     21905
                ...  
ktAc39_WaTI        71
ivXS0Fnzpns        44
KwWz3wQtV5Y        42
zZGtLXxQhjQ        25
FUzLx1l37RE        25
Name: count, Length: 310, dtype: int64

In [16]:
video_title = df_metadata[df_metadata.id == 'svOc4Ki68_U']['title'].values[0]
print(video_title)
print("It's a debate" if "debate" in video_title.lower() else "It's not a debate")

HEATED DEBATE: Cornel West, Alan Dershowitz spar over Israel-Hamas war
It's a debate


In [17]:
video_id = 'svOc4Ki68_U'
video_comments =  df_comments[df_comments['video_id'] == video_id]

In [33]:
%%time
comment_sentiments = []
for i, comment in enumerate(tqdm(video_comments['text'].tolist())):
    try:
        comment_sentiments.append(sentiment_pipeline(comment))
    except:
        print(f'Problem with comment #{i}')
        comment_sentiments.append([{'label': 'UNKNOWN', 'score': 1.}])

  0%|          | 0/24391 [00:00<?, ?it/s]

C:\Users\harri\anaconda3\Lib\site-packages\transformers\pipelines\base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


Problem with comment #4259
Problem with comment #5677
Problem with comment #8797
Problem with comment #10560
Problem with comment #10563
Problem with comment #10564
Problem with comment #10577
Problem with comment #12412
Problem with comment #14575
Problem with comment #14658
Problem with comment #18113
Problem with comment #21035
Problem with comment #22076
Problem with comment #24162
CPU times: total: 1min 55s
Wall time: 2min 12s


In [34]:
video_comments['text'].tolist()[8797]

'ليسن فاكس نيوز اذا كنت تفهم الصحافه واذا كنت تفهم القراءه والكتابه هل تعلم ان اسرائيل محتله الاراضي الفلسطينيه اكثر من 75 عام تعلم ان اسرائيل تقتل الفلسطينيين يوميا وتقول ان حماس هي ارهابيه حماس نينجا جو هما فلسطينيين يدافعون عن عرضهم وعن كرامتهم وعن حقوقهم من حق كل فلسطينيه حماس اي انسان فلسطيني ان يدافع على حقوقه في الدول العربيه والاسلاميه هي ليست بالخمسينات احنا فهمنا الكثير منكم وعلى عبر قناتكم فاكس نيوز تعلمنا الكثير العنصريه حقكم من الاضطهاد من الظلم من الكهنوت انت من حق اي فلسطيني ان يدافع وان يقتل الجيش الاسرائيلي كله وفي مثل بيقول بالعربي اي شيء يؤخذ منك بالقوه ما يعاد الا بالقوه نحن نعرف الكثير ان اسرائيل هي ولايه من الولايات المتحده الامريكيه تدعمها بالمال بالاسلحه وتقتلون شعبنا العزيز فجاء الحق وزهق الباطل ان الباطل كان زهوقا والنصر والعزه لابناء فلسطين ونحن واقفين معهم بالمال والسلاح والرجال وفي كل شيء رغم انفكم'

In [24]:
print(len(comment_sentiments))
comment_sentiments[0]

24377


[{'label': 'NEGATIVE', 'score': 0.99345463514328}]

In [35]:
# Drop low-confidence (<80%) classifications
sentiment_counter = Counter([s[0]['label'] for s in comment_sentiments if s[0]['score'] > 0.8])

In [36]:
print(sum(sentiment_counter.values()))
sentiment_counter.most_common()

22886


[('NEGATIVE', 17094), ('POSITIVE', 5778), ('UNKNOWN', 14)]

In [38]:
video_comments['sentiment'] = [s[0]['label'] if s[0]['score'] > 0.8 else 'UNCERTAIN' for s in comment_sentiments]

C:\Users\harri\AppData\Local\Temp\ipykernel_22936\724003131.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  video_comments['sentiment'] = [s[0]['label'] if s[0]['score'] > 0.8 else 'UNCERTAIN' for s in comment_sentiments]


In [40]:
video_comments['sentiment'].value_counts()

sentiment
NEGATIVE     17094
POSITIVE      5778
UNCERTAIN     1505
UNKNOWN         14
Name: count, dtype: int64

In [44]:
video_comments[video_comments['sentiment'] == 'POSITIVE']['text'].head()

823034                                                                                                                                                    Doctor West!!!! One of the greatest humans of our time.
823036                                                                                                                      Alan was Epsteins best friend…. If you support Israel you support evil child grapeist
823037                                                                                                                                             Fox news all of your media outlets are sellouts for the dollar
823040                                                                                                                     Dr. West always keeps it real keep going brother speaking that truth we got your back.
823042    Even Biden took his words back, his accusations or beheaded babies. Yet they feel comfortable to us it as a fact and win this way and argument or sham